In [ ]:
# Removes large-scale background gradients from the input flc files, and equalizes the overall
# background levels between chips

from astropy.convolution import Gaussian2DKernel
from astropy.io import fits
from astropy.stats import gaussian_fwhm_to_sigma, SigmaClip, sigma_clip
import glob
import numpy as np
from photutils import Background2D, detect_sources, detect_threshold, MedianBackground


################################# USER INPUTS #################################

# The files to correct
files = glob.glob('./*flc.fits')

# Only those files whose gradient is larger than this threshold will be corrected.
# Set to None to correct all files regardless of the measured gradient.
gradient_threshold = 5.0

# The box size to use when creating the 2D background image
box_size = (128, 128)

# Option to mask sources when finding the background gradient/pedestal level
mask_sources = True

###############################################################################


# Remove those files with no gradient from the processing list
if gradient_threshold:
    files_to_process = []
    for f in files:
        diffs = []
        for ext in [1, 4]:
            data = fits.getdata(f, ext)
            left, _,  _, right = np.split(data, 4, axis=1)
            clipped_left = sigma_clip(left, sigma=3, maxiters=5)
            clipped_right = sigma_clip(right, sigma=3, maxiters=5)
            med_left = np.nanmedian(clipped_left.data[clipped_left.mask==False])
            med_right = np.nanmedian(clipped_right.data[clipped_right.mask==False])
            diffs.append(abs(med_left - med_right))
        diffs = np.array(diffs)
        if len(diffs[diffs > gradient_threshold]) > 0:
            files_to_process.append(f)
else:
    files_to_process = files


# STEP 1: Remove the large-scale 2D background gradient from each chip
print('STEP 1: Removing 2D gradients from input FLCs...')
for f in files_to_process:
    basename = os.path.basename(f)
    print('Working on {}:'.format(basename))
    h = fits.open(f)
    for ext in [1,4]:
        print('\tWorking on extension {}:'.format(ext))
        data_orig = np.copy(h[ext].data)
        data = h[ext].data

        # Subtract off median
        clipped = sigma_clip(data, sigma=3, maxiters=5)
        data = data - np.nanmedian(clipped.data[clipped.mask==False])

        # Find sources in the gradient-removed image
        if mask_sources:
            print('\tMaking source segmap...')
            s = SigmaClip(sigma=3.)
            bkg_estimator = MedianBackground()
            bkg = Background2D(data, box_size=box_size, filter_size=(10, 10), 
                               sigma_clip=s, bkg_estimator=bkg_estimator, exclude_percentile=15.0)
            skydark = bkg.background
            data_flat = data_orig - skydark
            threshold = detect_threshold(data_flat, nsigma=0.75)
            sigma = 3.0 * gaussian_fwhm_to_sigma
            kernel = Gaussian2DKernel(sigma, x_size=3, y_size=3)
            kernel.normalize()
            segm = detect_sources(data_flat, threshold, npixels=5, filter_kernel=kernel)
            segmap = segm.data
            fits.writeto(f.replace('_flc.fits', '_segmap_ext{}.fits'.format(ext)), 
                         segmap, overwrite=True)
        else:
            segmap = np.zeros(data.shape).astype(int)
        
        # Find the background gradient, incorporating the source mask
        print('\tFinding the background gradient...')
        s = SigmaClip(sigma=3.)
        bkg_estimator = MedianBackground()
        mask = (segmap > 0)
        bkg = Background2D(data, box_size=box_size, filter_size=(10, 10), 
                           sigma_clip=s, bkg_estimator=bkg_estimator, mask=mask, exclude_percentile=15.0)
        skydark = bkg.background
        fits.writeto(f.replace('_flc.fits', '_bkg_ext{}.fits'.format(ext)), 
                     skydark, overwrite=True)

        # Subtract the background gradient from the original image
        data_new = data_orig - skydark
        h[ext].data = data_new.astype('float32')

    h.writeto(f, overwrite=True)
    h.close()
    print('Finished removing background gradient from {}'.format(basename))


# STEP 2: equalize the overall background/pedestal level between chips to the average of the two chips
print('\nSTEP 2: Equalizing overall background levels in the input FLCs...')
for f in files_to_process:
    basename = os.path.basename(f)
    print('Working on {}:'.format(basename))
    h = fits.open(f)
    
    # Find the overall background levels in each chip
    background_levels = []
    for ext in [1,4]:
        data = np.copy(h[ext].data)
        
        # Mask sources using the previous segmap
        if mask_sources:
            segmap = fits.getdata(f.replace('_flc.fits', '_segmap_ext{}.fits'.format(ext)))
        else:
            segmap = np.zeros(data.shape).astype(int)
        data[segmap > 0] = np.nan
        
        # Calculate the background level in the chip
        clipped = sigma_clip(data, sigma=3, maxiters=5)
        background_levels.append(np.nanmedian(clipped.data[clipped.mask==False]))
        print('\tBackground in ext {} = {:0.5f}'.format(ext, np.nanmedian(clipped.data[clipped.mask==False])))
    
    # Equalize the background levels of the two chips to the average of the two
    avg_bkg = np.mean(background_levels)
    ext1_orig = np.copy(h[1].data)
    ext1_diff = background_levels[0] - avg_bkg
    ext1_new = ext1_orig - ext1_diff
    h[1].data = ext1_new.astype('float32')
    ext4_orig = np.copy(h[4].data)
    ext4_diff = background_levels[1] - avg_bkg
    ext4_new = ext4_orig - ext4_diff
    h[4].data = ext4_new.astype('float32')
    
    # Write out the final calibrated file
    h.writeto(f, overwrite=True)
    h.close()
    print('Finished equalizing background levels between chips in {}'.format(basename))
